# Рекомендация тарифов

**Цель проекта**: разработка модели классификации тарифов для оператора сотовой связи в зависимости от поведения абонентов на основании имеющихся данных об абонентах, которые уже используют целевые тарифы. Минимально необходимая точность прогнозирования модели - 75%. 

В нашем распоряжении следующая информация о поведении пользователей за месяц:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).


**Основные этапы проекта**:

- Изучение предоставленных данных
- Разбиение данных на выборки
- Выбор и построение моделей классификации
- Проверка моделей на тестовой выборке
- Поверка моделей на адекватность
- Общий вывод и рекомендации

## Изучение предоставленных данных

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier

In [4]:
tarifs = pd.read_csv('/datasets/users_behavior.csv')
print(tarifs.info())
tarifs.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Проверим таблицу на дубликаты:

In [5]:
tarifs.duplicated().sum()

0

Для удобства переведём столбцы с **количеством звонков** и **отправленных сообщений** в целочисленный тип:

In [6]:
tarifs['calls'] = tarifs['calls'].astype(int)
tarifs['messages'] = tarifs['messages'].astype(int)
tarifs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null int64
minutes     3214 non-null float64
messages    3214 non-null int64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


### Вывод
В предоставленной таблице 3 214 строк и 5 столбцов.

Пропусков в данных нет, дубликатов тоже. Предобработка данных не требуется, однако для удобства анализа мы привели столбцы с данными о количестве звонков и отправленных сообщений к целочисленному типу.

## Разбиение данных на выборки

Сперва выделим целевой признак (выбранный абонентом тариф - "Ультра" или "Смарт") и влияющие на него признаки (остальные столбцы в таблице):

In [7]:
target = tarifs['is_ultra']
features = tarifs.drop('is_ultra', axis = 1)
print(target.shape)
print(features.shape)

(3214,)
(3214, 4)


Для анализа выделим валидационную и тестовую выборки, по 20% наблюдений в каждой. Оставшиеся 60% данных будут использованы в обучении моделей.

In [8]:
# сперва выделим 60% обучающих данных:
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features, target, test_size = 0.4, random_state = 12345)
# затем оставшиеся 40% разделим на две выборки по 20%:
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size = 0.5, 
    random_state = 12345)
print('Размер тренировочной выборки:', target_train.shape)
print('Размер валидационной выборки:', target_valid.shape)
print('Размер тестовой выборки:', target_test.shape)

Размер тренировочной выборки: (1928,)
Размер валидационной выборки: (643,)
Размер тестовой выборки: (643,)


### Вывод
Мы разбили наш набор данных на тренировочный (1928 наблюдений), на котором и будем обучать модели, валидационный и тестовый (по 643 наблюдения), на которых будем проверять качество моделей. 

## Исследование моделей

Поскольку целевой признак в нашем случае - категориальная переменная, которая принимает два значения, перед нами стоит задача классификации. Для этого будем использовать следующие модели:
- дерево решений
- случайный лес
- логистическую регрессию.

Начнём с построения модели **дерева решений**. Подберём оптимальную при различной максимальной глубине дерева - от 1 до 10. Для оценки качества модели будем использовать точность прогнозирования (accuracy) модели на валидационной выборке.

In [9]:
best_tree_model = None
best_tree_accuracy = 0
best_tree_depth = 0
for depth in range(1,11):
    model = DecisionTreeClassifier(random_state = 1, max_depth = depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predictions)
    if accuracy > best_tree_accuracy:
        best_tree_model = model
        best_tree_accuracy = accuracy
        best_tree_depth = depth
print('Лучшая модель дерева решений показала точность прогнозирования ', 
      best_tree_accuracy.round(3)*100, 
      '% при максимальной глубине дерева, равной ', best_tree_depth, sep='')

Лучшая модель дерева решений показала точность прогнозирования 78.5% при максимальной глубине дерева, равной 3


Следующей построим модель **случайного леса**. В качестве гиперпараметров зададим диапазон максимальной глубины дерева от 1 до 10, количество деревьев - от 5 до 60 с шагом 5.

In [10]:
best_forest_model = None
best_forest_accuracy = 0
best_forest_depth = 0
best_forest_est = 0
for depth in range(1,11):
    for est in range(5,61,5):
        model = RandomForestClassifier(random_state = 1, max_depth = depth, n_estimators = est)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predictions)
        if accuracy > best_tree_accuracy:
            best_forest_model = model
            best_forest_accuracy = accuracy
            best_forest_depth = depth
            best_forest_est = est
print('Лучшая модель случайного леса показала точность прогнозирования ', 
      (best_forest_accuracy*100).round(1), 
      '% при максимальной глубине дерева, равной ', best_forest_depth, 
      ' и количестве деревьев, равном ', best_forest_est, sep='')

Лучшая модель случайного леса показала точность прогнозирования 80.1% при максимальной глубине дерева, равной 10 и количестве деревьев, равном 60


Осталось построить модель **логистической регрессии**:

In [11]:
model = LogisticRegression(random_state = 1)
model.fit(features_train, target_train)
logistic_accuracy = model.score(features_valid, target_valid)
print('Модель логистической регрессии показала точность прогнозирования ', 
      logistic_accuracy.round(3)*100, '%', sep = '')

Модель логистической регрессии показала точность прогнозирования 75.9%


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Вывод
Мы построили 3 вида моделей для классификации пользователей по выбранным тарифам в зависимости от их поведения в течение месяца, которые показали следующую точность прогнозирования на валидационной выборке:
- дерево решений с максимальной глубиной 3: 78,5%;
- случайный лес с максимальной глубиной дерева 10 и количеством деревьев 60: 80,1%;
- логистическая регрессия: 75,9%.

Как видим, на валидационной выборке лучше всего работает модель случайного леса. Проверим каждую модель на тестовой выборке.

## Проверка модели на тестовой выборке

Начнём с модели **дерева решений**, максимальная глубина дерева - 3.

In [12]:
tree_model = best_tree_model
tree_model.fit(features_train, target_train)
tree_predictions = tree_model.predict(features_test)
tree_test_accuracy = accuracy_score(target_test, tree_predictions)
print('Модель дерева решений на тестовой выборке показала точность прогнозирования ', 
      tree_test_accuracy.round(3)*100, '%', 
      sep='')

Модель дерева решений на тестовой выборке показала точность прогнозирования 77.9%


Модель **случайного леса**:

In [13]:
forest_model = best_forest_model
forest_model.fit(features_train, target_train)
forest_predictions = forest_model.predict(features_test)
forest_test_accuracy = accuracy_score(target_test, forest_predictions)
print('Модель случайного леса на тестовой выборке показала точность прогнозирования ',
      forest_test_accuracy.round(3)*100, '%', 
      sep='')

Модель случайного леса на тестовой выборке показала точность прогнозирования 80.4%


Модель **логит-регрессии**:

In [14]:
logit_model = LogisticRegression(random_state = 1)
logit_model.fit(features_train, target_train)
logistic_test_accuracy = logit_model.score(features_test, target_test)
print('Модель логистической регрессии на тестовой выборке показала точность прогнозирования ',
      logistic_test_accuracy.round(3)*100, '%', 
      sep='')

Модель логистической регрессии на тестовой выборке показала точность прогнозирования 74.0%


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Вывод

На тестовой выборке модели дерева решений и логит-регрессии получили результаты точности прогнозирования ниже, чем на валидационной: 77,9% и 74% соответственно. А модель случайного леса напротив, повысила точность прогнозирования: 80,4% на тестовой выборке против 79,8% на валидационной. 

## Проверка модели на адекватность

Воспользуемся функцией classification_report библиотеки sklearn для проверки качества выбранных моделей, которая рассчитывает для каждого класса и модели в целом следующие показатели: 

*Precision (точность)* - показывает, какая доля объектов, выделенных классификатором в данный класс, действительно принадлежат к данному классу.

*Recall (полнота)*  - показывает, какая доля объектов из этого класса выделена классификатором как принадлежащие к данному классу.

*F-мера* - гармоническое среднее точности и полноты.

Для лучшей модели **дерева решений** classification_report выглядит следующим образом:

In [15]:
print(classification_report(target_test, tree_predictions))

              precision    recall  f1-score   support

           0       0.79      0.93      0.85       440
           1       0.74      0.46      0.57       203

    accuracy                           0.78       643
   macro avg       0.77      0.69      0.71       643
weighted avg       0.77      0.78      0.76       643



Для лучшей модели **случайного леса**:

In [16]:
print(classification_report(target_test, forest_predictions))

              precision    recall  f1-score   support

           0       0.82      0.92      0.87       440
           1       0.76      0.56      0.64       203

    accuracy                           0.80       643
   macro avg       0.79      0.74      0.75       643
weighted avg       0.80      0.80      0.79       643



Для **логит-регрессии**.

In [17]:
print(classification_report(target_test, logit_model.predict(features_test)))

              precision    recall  f1-score   support

           0       0.73      0.98      0.84       440
           1       0.82      0.23      0.36       203

    accuracy                           0.74       643
   macro avg       0.78      0.60      0.60       643
weighted avg       0.76      0.74      0.69       643



Сравним результаты построенных моделей с константной, полученной с помощью DummyClassifier, в качестве стратегии будет использована мода целевого признака.

In [36]:
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=0)
dummy_clf.fit(features_train, target_train)
dummy_clf.predict(features_test)
print('Точность предсказания фиктивного классификатора: ',
      dummy_clf.score(features_test, target_test).round(2)*100, '%', sep='')

Точность предсказания фиктивного классификатора: 68.0%


Сравнение протестированных моделей с фиктивным классификатором свидетельствует об адекватности всех 3х рассмотреных моделей, поскольку их accuracy превышает точность фиктивного классификатора (однако, не намного).

Как видим из отчётов, все три модели имеют достаточно низкие ошибки классификации: средневзвешенный показатель F1 для дерева решений равен 76%, для случайного леса - 79%, для логистической регрессии - 69%. Точность и полнота для всех моделей превышает 73%. И снова можно убедиться в том, что качество модели случайного леса выше, чем у двух других.

# Вывод

Для задачи классификации тарифов в зависимости от поведения пользователя было построено 3 модели:
- дерево решений;
- случайный лес;
- логит-регрессия.

Лучшую точность прогнозирования как на валидационной, так и на тестовой выборке, показала модель случайного леса со следующими параметрами: количество деревьев - 60, максимальная высота дерева - 10. Бизнесом поставлена задача - достичь точности прогнозирования на тестовой выборке не менее 75%. Выбранная модель верно предсказала выбор тарифа пользователем на тестовых данных в 80,4% случаев.

Все модели проверены с помощью метрик качества и имеют досточно низкий процент ошибок классификатора. Сравнение с фиктивным классификатором свидетельствует об адекватности всех рассмотренных моделей.

**Таким образом, можно рекомендовать оператору использовать выбранную модель случайного леса для выработки предложений клиентам, использующим архивные тарифы.**